参考: https://www.tensorflow.org/programmers_guide/variables

## Variable
在 TensorFlow 中，Variable是表示共享、持久状态的最佳方式。它通过 tf.Variable 对象来表示。tf.Variable 表示一个可以通过操作(ops)来改变其值的 Tensor.

对 tf.Variable 的个性可以跨越多个 Session, 因为可以在不同的 Worker中个性同一个 Variable。

创建 Variable 的最佳方式是调用 tf.get_variable 方法。

In [ ]:
my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32, 
  initializer=tf.zeros_initializer)

其中，dtype 默认为 tf.float32, initializer 默认为`tf.glorot_uniform_initializer`

也可以将一个 Tensor 作为 initializer, 这时，不可以再指定 shape

In [ ]:
other_variable = tf.get_variable("other_variable", dtype=tf.int32, 
  initializer=tf.constant([23, 42]))

## Variable Collections

TensorFlow 通过 collections 提供对变量的全局访问。默认情况下，对于变量维护两个列表，即`tf.GraphKeys.GLOBAL_VARIABLES`和`tf.GraphKeys.TRAINABLE_VARIABLES`

如何不想让一个 Variable 变得 trainnable, 可以通过如下方式指定

In [ ]:
my_local = tf.get_variable("my_local", shape=(), 
collections=[tf.GraphKeys.LOCAL_VARIABLES])

# 也可以这样
my_non_trainable = tf.get_variable("my_non_trainable", 
                                   shape=(), 
                                   trainable=False)

也可以创建和维护自己的 collection

In [ ]:
tf.add_to_collection("my_collection_name", my_local)
# 获取之
tf.get_collection("my_collection_name")

可以指定将变量放置在指定的设备上

In [ ]:
with tf.device("/gpu:1"):
  v = tf.get_variable("v", [1])

` tf.train.replica_device_setter`可以用来将变量自动放置到参数服务器上。

In [11]:
cluster_spec = {
    "ps": ["ps0:2222", "ps1:2222"],
    "worker": ["worker0:2222", "worker1:2222", "worker2:2222"]}
with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):
    v = tf.get_variable("v", shape=[20, 20])  # this variable is placed 
                                            # in the parameter server
                                            # by the replica_device_setter

(<tf.Variable 'a:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'a_1:0' shape=(1,) dtype=float32_ref>)

## 变量的初始化

所有的变量在使用之前都必须初始化。如果你使用的是底层的 API， 初始化需要显示指定。如果使用的是上层的 API，比如`tf.contrib.slim`, `tf.estimator.Estimator`, `Keras`, 变量的初始化则是自动进行的。

>TODO: Explicit initialization is otherwise useful because it allows you not to rerun potentially expensive initializers when reloading a model from a checkpoint as well as allowing determinism when randomly-initialized variables are shared in a distributed setting.

如果要一次性初始化所有可训练的变量，则在开始训练之前执行`tf.global_variables_initializer()`即可。

### 注意 
** `tf.global_variables_initializer()` 并不保证初始化是有序的！**

In [ ]:
# 对 tf.GraphKeys.GLOBAL_VARIABLES 中的所有变量进行初始化
session.run(tf.global_variables_initializer())
# Now all variables are initialized.

如果需要初始化指定的变量:

In [ ]:
session.run(my_variable.initializer)

也可以这样查询哪些变量还没有被初始化

In [ ]:
print(session.run(tf.report_uninitialized_variables()))

不过，下面这段话没看懂:
```python
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = tf.get_variable("w", initializer=v.initialized_value() + 1)
```

## 使用变量

In [1]:
import tensorflow as tf
sess = tf.Session()

要使用变量的值，就将它当作一个普通的 Tensor 就好了:
```python
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = v + 1  # w is a tf.Tensor which is computed based on the value of v.
           # Any time a variable is used in an expression it gets automatically
           # converted to a tf.Tensor representing its value.
```

给变量赋值

In [2]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)

sess.run(tf.global_variables_initializer())
print sess.run([assignment, v])

[1.0, 1.0]


`v.assign_add`返回的是一个包含变量赋值后的值的 Tensor.

>对于官网上下面这段代码我也不是很理解。我的理解是， w 应该保存的是固定的值？可是实际运行来看，assignment, assignment2和 w 的值是一样的，指向的应该是同一个 Tensor.

In [2]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
with tf.control_dependencies([assignment]):
  w = v.read_value()  # w is guaranteed to reflect v's value after the
                      # assign_add operation.

In [4]:
sess.run(tf.global_variables_initializer())
sess.run(w)

1.0

In [6]:
assignment2 = v.assign_add(1)

In [14]:
sess.run([assignment, assignment2, w])

[14.0, 14.0, 14.0]

## 变量域

在 TensorFlow 中，变量的名称也是以类似目录的形式组织的, 并支持多级嵌套。例如，在以下代码中，变量的名称遵从 `scope 名称/变量名称:序号` 的形式。

In [15]:
#import tensorflow
import tensorflow as tf
#open a variable scope named 'scope1'
with tf.variable_scope("scope1"):
    #open a nested scope name 'scope2'
    with tf.variable_scope("scope2"):
        #add a new variable to the graph
        var=tf.get_variable("variable1",[1])
#print the name of variable
print(var.name)

scope1/scope2/variable1:0


在`tf.variable_scope`中调用 `get_variable`即可以用来创建变量，也可以用来复用已有的变量。

## 权重共享

In [ ]:
#import tensorflow
import tensorflow as tf
#open a variable scope named 'scope1'
with tf.variable_scope("scope1"):
    #declare a variable named variable1
    var1 = tf.get_variable("variable1",[1])
    #declare another variable with same name
    var2=tf.get_variable("variable1",[1])

运行以上代码会抛异常:
```
ValueError: Variable scope1/variable1 already exists, disallowed. Did you mean to set reuse=True in VarScope? 
```

因为已经在 scope1中定义了一个名为 variable1的变量，而且 variable_scope 的 reuse flag 默认是 False 的。

要共享变量，可以在调用`variable_scope`时，将其 reuse 参数设置为 True

In [6]:
#import tensorflow
import tensorflow as tf
#open a variable scope named 'scope1'
with tf.variable_scope("scope1", reuse = True):
    #declare a variable named variable1
    var1 = tf.get_variable("variable1",[1])
    #declare another variable with same name
    var2=tf.get_variable("variable1",[1])
    assert var1 == var2

另一种方法是显示调用

In [1]:
#import tensorflow
import tensorflow as tf
#open a variable scope named 'scope1'
with tf.variable_scope("scope1"):
    #declare a variable named variable1
    var1 = tf.get_variable("variable1",[1])
    tf.get_variable_scope().reuse_variables()
    assert tf.get_variable_scope().reuse == True
    #declare another variable with same name
    var2=tf.get_variable("variable1",[1])
    assert var1 == var2

以上代码会正常运行，且不会抛任何异常。

## name_scope vs variable_scope

In [1]:
import tensorflow as tf
with tf.variable_scope("sp1"):
    v1 = tf.get_variable("v1", [1])
    v2 = v1 + 2
v1, v2

(<tf.Variable 'sp1/v1:0' shape=(1,) dtype=float32_ref>,
 <tf.Tensor 'sp1/add:0' shape=(1,) dtype=float32>)

In [1]:
import tensorflow as tf
with tf.name_scope("sp1"):
    v1 = tf.get_variable("v1", [1])
    v2 = v1 + 2
v1, v2

(<tf.Variable 'v1:0' shape=(1,) dtype=float32_ref>,
 <tf.Tensor 'sp1/add:0' shape=(1,) dtype=float32>)